In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory, ConversationSummaryMemory, ConversationSummaryBufferMemory, ConversationKGMemory
from langchain.chains import LLMChain
from langchain.schema.runnable import RunnablePassthrough

In [3]:
llm = ChatOpenAI(temperature=0.1)
memory = ConversationBufferMemory(return_messages=True) # keep all messages in memory
memory = ConversationBufferWindowMemory(
    k=4, # k=4: number of recent messages to keep in memory
    return_messages=True # return_messages=True: used for chat model
)
memory = ConversationSummaryMemory(llm=llm) # summarize the conversation using llm
memory = ConversationSummaryBufferMemory( # summarize the old conversation after token limit (mix of both)
    llm=llm,
    max_token_limit=150,
    return_messages=True
)
memory = ConversationKGMemory( # use knowledge graph to extract entities and relationships
    llm=llm,
    return_messages=True,
)

"""
try using 
def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})
def get_history():
    return memory.load_memory_variables({})
"""

'\ntry using \ndef add_message(input, output):\n    memory.save_context({"input": input}, {"output": output})\ndef get_history():\n    return memory.load_memory_variables({})\n'

In [7]:
# connect memory to chain

## off-the-shelf chain
llm = ChatOpenAI(temperature=0.1)

### text
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=50,
    memory_key="history" # used to load memory variables in llmchain
)
template = """
    You are a helpful AI talking to a human.
    {history}
    Human:{question}
    You:
    """

chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(template),
    memory=memory,
    verbose=True, # debugging
)
chain.predict(question="My name is Darren.")
chain.predict(question="What is my name?")

### chat
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=50,
    memory_key="history", # used to load memory variables in llmchain
    return_messages=True
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)
chain = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory,
    verbose=True,
)
chain.predict(question="My name is Darren.")
chain.predict(question="What is my name?")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: My name is Darren.

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: My name is Darren.
AI: Hello Darren! How can I assist you today?
Human: What is my name?

> Finished chain.


'Your name is Darren.'

In [4]:
## customized chain using lcel
### chat
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=50,
    memory_key="history", # used to load memory variables in llmchain
    return_messages=True
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)
def load_memory(_):
    return memory.load_memory_variables({})["history"]
def invoke_chain(question):
    answer = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": answer.content},
    )
    print(answer)
chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm # output of load_memory is assigned to history when invoked
invoke_chain("My name is Darren.")
invoke_chain("What is my name?")

content='Hello Darren! How can I assist you today?'
content='Your name is Darren.'
